# **Sequential Hyperparameter Search and Sensitivity Analysis of Color Channels**

This notebook implements sequential hyperparameter search, sensitivity analysis of the RGB color channel, and systematic experiment tracking using Weights & Biases (wandb).

Instead of performing a full grid search (computationally expensive), a sequential strategy is used:  
At each stage:
- Only one hyperparameter is varied
- All other hyperparameters remain fixed
- The best-performing value is selected
- That value is fixed before moving to the next stage

This approach significantly reduces computational cost while maintaining controlled experimentation.

In [ ]:
import tensorflow as tf
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import wandb
from wandb.integration.keras import WandbMetricsLogger

from tools import load_images_with_labels, calculate_metrics, evaluate_model

## Load data

In [ ]:
#Image dimensions
width = 540
height = 960

#Path where images are stored and organized by class
path = '../data/burn_images/'

#Channel selection for sensitivity analysis
channels = ['green', 'blue'] 
n_channels = len(channels) #Number of input channels

In [ ]:
#Load images with their corresponding labels using only selected channels
X, y = load_images_with_labels(path=path, channels=channels)
print(X.shape, y.shape)

In [ ]:
#Split the dataset into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

print('Shape of X_train:', X_train.shape)
print('Shape of X_val:', X_val.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of y_val:', y_val.shape)

## Function to change hyperparameters

This function receives a hyperparameter and a list of hyperparameters, which are tested one by one in the model.  

In [ ]:
#Dictionary containing the base hyperparameter values
base_hyperparams = {
    'conv_layers': 3,
    'filters_layer_1': 32,
    'filters_layer_2': 32,
    'filters_layer_3': 64,
    'kernel_size': 3,
    'strides': 2,
    'dense_layers': 1,
    'dense_units_1': 64,
    'dense_units_2': 64,
    'dense_units_3': 64,
    'dropout_rate': 0.4,
    'batch_size': 32
}

In [ ]:
def change_hyperparam(config_dict, hyperparam, hyperparams_list):
    '''
    Function that changes the value of a hyperparameter in a dictionary and generates a run name.
    That is, it modifies a single hyperparameter while keeping the others constant.

    Parameters:
    - config_dict (dict): Dictionary containing the model parameters.
    - hyperparam (str): Key of the parameter to be changed.
    - hyperparams_list (list): List of values to assign to the parameter.

    Yields:
    - tuple: A run name (str) and the updated dictionary (dict).
    If the parameter is not found in the dictionary, it prints an error message and returns None.
    '''
    for value in hyperparams_list:
        #Modify only the target hyperparameter
        if hyperparam in config_dict:
            config_dict[hyperparam] = value
        else:
            print(f'Hyperparameter {hyperparam} not found in dictionary')
            return None
        
        #Build experiment run name
        run_name = (f"cl:{config_dict['conv_layers']}, " 
                    f"fl1:{config_dict['filters_layer_1']}, ")
        
        if config_dict['conv_layers'] >= 2: 
            run_name += f"fl2:{config_dict['filters_layer_2']}, "
        if config_dict['conv_layers'] >= 3: 
            run_name += f"fl3:{config_dict['filters_layer_3']}, "

        run_name += (f"ks:{config_dict['kernel_size']}, "
                     f"st:{config_dict['strides']}, "
                     f"dl:{config_dict['dense_layers']}, "
                     f"du1:{config_dict['dense_units_1']}, ")
        
        if config_dict['dense_layers'] >= 2:
            run_name += f"du2:{config_dict['dense_units_2']}, "  
        if config_dict['dense_layers'] >= 3:
            run_name += f"du3:{config_dict['dense_units_3']}, "  

        run_name += (f"dr:{config_dict['dropout_rate']}, "
                     f"bs:{config_dict['batch_size']}")

        yield run_name, config_dict

In [ ]:
#Create generator of run names and configurations
hyperparam_generator = change_hyperparam(base_hyperparams, 'batch_size', [32])

### From here, execute to evaluate the remaining values in the list

In [ ]:
#Retrieve the next hyperparameter configuration from the generator
run_name, hyperparams = next(hyperparam_generator)
print(run_name)

## CNN model construction

The architecture is dynamically built based on the selected hyperparameters.

In [ ]:
#First convolutional block (includes input shape)
model_architecture = [layers.Conv2D(filters=hyperparams['filters_layer_1'], 
                                    kernel_size=(hyperparams['kernel_size'], hyperparams['kernel_size']), 
                                    strides=(hyperparams['strides'], hyperparams['strides']),
                                    input_shape=(height, width, n_channels)),
                      layers.BatchNormalization(),
                      layers.Activation('relu'),
                      layers.MaxPooling2D(pool_size=(2, 2))]

#Remaining convolutional layers (added according to the value of conv_layers)
if hyperparams['conv_layers'] > 1:
    model_architecture += [layers.Conv2D(filters=hyperparams['filters_layer_2'], 
                                         kernel_size=(hyperparams['kernel_size'], hyperparams['kernel_size']),
                                         strides=(hyperparams['strides'], hyperparams['strides'])),
                           layers.BatchNormalization(),
                           layers.Activation('relu'),
                           layers.MaxPooling2D(pool_size=(2, 2))]

    if hyperparams['conv_layers'] > 2:
        model_architecture += [layers.Conv2D(filters=hyperparams['filters_layer_3'], 
                                             kernel_size=(hyperparams['kernel_size'], hyperparams['kernel_size']),
                                             strides=(hyperparams['strides'], hyperparams['strides'])),
                               layers.BatchNormalization(),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size=(2, 2))]
            
#Feature aggregation
model_architecture += [layers.GlobalAveragePooling2D()]

#Fully connected layers (added according to the value of dense_layers)
model_architecture += [layers.Dense(units=hyperparams['dense_units_1']),
                       layers.Activation('relu'),
                       layers.Dropout(rate=hyperparams['dropout_rate'])]

if hyperparams['dense_layers'] > 1:
    model_architecture += [layers.Dense(units=hyperparams['dense_units_2']),
                           layers.Activation('relu'),
                           layers.Dropout(rate=hyperparams['dropout_rate'])]

    if hyperparams['dense_layers'] > 2:
        model_architecture += [layers.Dense(units=hyperparams['dense_units_3']),
                               layers.Activation('relu'),
                               layers.Dropout(rate=hyperparams['dropout_rate'])]

#Output layer: probability of third-degree burn
model_architecture += [layers.Dense(units=1),
                       layers.Activation('sigmoid')]

#Instantiate the CNN model with the defined architecture
model = tf.keras.Sequential(model_architecture) 

In [ ]:
#Display the final model architecture
model.summary()

In [ ]:
#Model compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Create experiment in wandb

In [ ]:
#Build configuration dictionary for experiment tracking
configuracion = {'conv_layers': hyperparams['conv_layers'],
                 'filters_layer_1': hyperparams['filters_layer_1'],
                 'kernel_size': hyperparams['kernel_size'],
                 'strides': hyperparams['strides'],
                 'dense_layers': hyperparams['dense_layers'],
                 'dense_units_1': hyperparams['dense_units_1'],
                 'dropout_rate': hyperparams['dropout_rate'],
                 'batch_size': hyperparams['batch_size']}

if hyperparams['conv_layers'] > 1:
    configuracion['filters_layer_2'] = hyperparams['filters_layer_2']
    if hyperparams['conv_layers'] > 2:
        configuracion['filters_layer_3'] = hyperparams['filters_layer_3']

if hyperparams['dense_layers'] > 1:
    configuracion['dense_units_2'] = hyperparams['dense_units_2']
    if hyperparams['dense_layers'] > 2:
        configuracion['dense_units_3'] = hyperparams['dense_units_3']

In [ ]:
#Initialize W&B run
run = wandb.init(project='CNN_quemaduras_2', 
                 entity='frantorres14',
                 name='_'.join(channels), #run_name for hyperparameter search
                 config=configuracion)

config = wandb.config #Access w&b configuration object
wandb_logger = WandbMetricsLogger(config) #Callback to automatically log training metrics per epoch

## Model training

In [ ]:
#Stop training when the model stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, #Wait 10 epochs without improvement before stopping
    restore_best_weights=True #Restore weights from the epoch with the best val_loss
)

#Reduce the learning rate when the model stagnates
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, #Reduce the learning rate by half when there is no improvement
    patience=5, #Wait 5 epochs without improvement before reducing
    min_lr=1e-7 #Minimum allowed learning rate
)

In [ ]:
#Model training
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=config.batch_size,
                    validation_data=(X_val, y_val),
                    callbacks=[WandbMetricsLogger(), early_stopping, reduce_lr])

## Evaluation metrics

In [ ]:
#Evaluation metrics on the training set
CM_train, accuracy_train, precision_train, recall_train, f1_train = calculate_metrics(model, X_train, y_train)
evaluate_model(model, X_train, y_train, dataset='Training')

In [ ]:
#Evaluation metrics on the validation set
CM_val, accuracy_val, precision_val, recall_val, f1_val = calculate_metrics(model, X_val, y_val)
evaluate_model(model, X_val, y_val, dataset='Validation')

In [ ]:
#Record final performance metrics in wandb
wandb.log({'accuracy_train': accuracy_train,
           'precision_train': precision_train,
           'recall_train': recall_train,
           'f1_train': f1_train,
           'accuracy_val': accuracy_val,
           'precision_val': precision_val,
           'recall_val': recall_val,
           'f1_val': f1_val})

#Finish the experiment
run.finish()

### Hyperparameter search space

'conv_layers': [1, 2, 3]  
'filters_layer_k': [16, 32, 64]  
'kernel_size': [3, 5]  
'strides': [1, 2, 3]  
'dense_layers': [1, 2, 3]  
'dense_units_k': [32, 64, 128]  
'dropout_rate': [0.3, 0.4, 0.5]  
'batch_size': [16, 32, 64]  